In [ ]:
!pip install -qU weave

In [ ]:
import re

import requests
import wandb
from bs4 import BeautifulSoup

import weave

In [ ]:
wandb.init(project="groq-rag", job_type="upload-dataset")
weave.init(project_name="groq-rag")

In [ ]:
def remove_selected_substrings(text):

    def check_if_contains_target(x):
        targets = ["Comptt.", "Delhi", "All India"]
        return any(target in x for target in targets)

    def replace_func(match):
        if check_if_contains_target(match.group()):
            return ""
        else:
            return match.group()

    result = re.sub(r"\(([^()]|\([^()]*\))*\)", replace_func, text)
    return result

In [ ]:
def extract_qa(qa_pair):
    match = re.search(r"Question \d+\.(.*?)Answer:(.*)", qa_pair)
    if match:
        question = match.group(1).strip()
        answer = match.group(2).strip()
        return {
            "question": remove_selected_substrings(question),
            "answer": answer
        }
    return None

In [ ]:
def generate_question_bank(chapter_url: str, chapter_name: str):
    response = requests.get(chapter_url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    question_answer_pairs = []
    for p_tag in soup.find_all('p'):
        question_answer_pair = p_tag.get_text(strip=True)
        question_answer_pairs.append(question_answer_pair)
    marks = ""
    question_bank = []
    for qa_pair in question_answer_pairs:
        if "Short Answer Type Questions".lower() in qa_pair.lower():
            marks = "3-4"
        elif "Long Answer Type Questions".lower() in qa_pair.lower():
            marks = "5-6"
        elif "Question" in qa_pair and "Answer:" in qa_pair:
            extracted_qa = extract_qa(qa_pair)
            if extracted_qa is not None:
                extracted_qa["marks"] = marks
                extracted_qa["chapter_name"] = chapter_name
                question_bank.append(extracted_qa)
    return question_bank

In [ ]:
chapter_urls = [
    "https://www.learncbse.in/the-last-lesson-important-questions/",
    "https://www.learncbse.in/lost-spring-important-questions/",
    "https://www.learncbse.in/deep-water-important-questions/",
    "https://www.learncbse.in/the-rattrap-important-questions/",
    "https://www.learncbse.in/indigo-important-questions/",
    "https://www.learncbse.in/poets-and-pancakes-important-questions/",
    "https://www.learncbse.in/the-interview-important-questions/",
    "https://www.learncbse.in/going-places-important-questions/",
]

chapter_names = [
    "The Last Lesson",
    "Lost Spring Important",
    "Deep Water Important",
    "The Rattrap Importan",
    "Indigo Important",
    "Poets and Pancakes",
    "The Interview",
    "Going Places",
]

question_bank = []
for chapter_url, chapter_name in zip(chapter_urls, chapter_names):
    question_bank.extend(
        generate_question_bank(chapter_url, chapter_name)
    )

In [ ]:
dataset = weave.Dataset(
    name="flamingos-prose-question-bank", rows=question_bank
)
weave.publish(dataset)

In [ ]:
table = wandb.Table(
    columns=["chapter_name", "question", "answer", "marks"]
)

for data in question_bank:
    table.add_data(
        data["chapter_name"],
        data["question"],
        data["answer"],
        data["marks"]
    )

wandb.log({"question_bank/flamingos_prose": table})
wandb.finish()